In [107]:
from ipyleaflet import (
    Map,
    Polyline, Polygon, Rectangle, Circle, CircleMarker,
    GeoJSON
)

from ipywidgets import IntSlider, jslink

In [108]:
import matplotlib as mpl
import matplotlib.cm
import matplotlib.colors
import numpy as np
from math import floor

In [109]:
import json
import os

In [110]:
with open('providers-by-zip.json') as f:
    data = json.load(f)

max_providers = data['result']['max_num_providers']
n_todraw = len(data['result']['zip_codes'])

In [111]:
center = [37.8, -96.9]
zoom = 4

m = Map(center=center, zoom=zoom)

n_colors = 10

In [122]:
def make_colors(colormap=mpl.cm.Blues):
    data = np.linspace(0.0,1.0,n_colors)
    c = [mpl.colors.rgb2hex(d[0:3]) for d in colormap(data)]
    return c

In [123]:
colors = make_colors()

In [124]:
to_plot = {}

for r in data['result']['zip_codes']:
    num_providers = r['num_providers']
    color = colors[int(floor(n_colors * (float(num_providers) / max_providers))) - 1]
    
    r['properties'] = {'style': {'color':color, 'weight': 1, 'fillColor':color, 'fillOpacity':0.5}, 'num_providers': num_providers}
    
    to_plot[r["zip_code"]] = r

In [125]:
plot_list = []

In [126]:
for f in os.listdir("./json-zips/zip_codes_for_the_usa"):
    with open(os.path.join("./json-zips/zip_codes_for_the_usa", f)) as fdata:
        area = json.load(fdata)
        map_out(area)

In [127]:
def map_out(area):
    for feature in area['features']:
        zip_code = zip_fix(feature['properties']['ZIP'])
        try:
            draw_data(zip_code, feature)
        except:
            pass

In [128]:
def zip_fix(zip_code):
    zip_code = str(zip_code)
    while len(zip_code) < 5:
        zip_code = "0" + zip_code
    return zip_code

In [129]:
def draw_data(zip_code, feature):
    for k, v in to_plot[zip_code]['properties'].iteritems():
        feature['properties'][k] = v
    plot_list.append(feature)

In [130]:
data['features'] = plot_list

In [131]:
g = GeoJSON(data=data)
m.add_layer(g)

In [132]:
m

In [106]:
zoom_slider = IntSlider(description='Zoom', min=2, max=15, value=zoom)
jslink((zoom_slider, 'value'), (m, 'zoom'))
zoom_slider